Note:

this file contains two main part

1. find proximity(time) of something(nearest ptv stop). This requires a local running ORS server, which might not be reproducable

2. join to cleaned data

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy as np
import requests
import json
from tqdm.auto import tqdm
tqdm.pandas()  # "tqdm>=4.9.0"
school_path = "../data/raw/school_zones/school_zones.csv"
ptv_path = "../data/raw/ptv_zones/ptv_zones.csv"

/home/even/repo/project-2-group-real-estate-industry-project-2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Rental

In [27]:
path = "../data/raw/oldlistings_rent_2.json"
property_df = pd.read_json(open(path, "r"))

In [28]:
property_df

,lat,lng,rented_prices,address,meta_data
0,-37.813730,144.955580,"[{'date': 'August 2023', 'price': '$800 per we...","201/560 LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
1,-37.813730,144.955580,"[{'date': 'August 2023', 'price': '$720 per we...","1702/560 LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
2,-37.810280,144.956670,"[{'date': 'August 2023', 'price': '$830 per we...","2007/228 ABECKETT STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
3,-37.813590,144.955720,"[{'date': 'August 2023', 'price': '$660 per we...","901/21 HEALEYS LANE, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
4,-37.813700,144.953930,"[{'date': 'August 2023', 'price': '$830 per we...","3801/601 LITTLE LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
...,...,...,...,...,...
539050,-37.840959,145.032227,"[{'date': 'November 2007', 'price': '$575 pw'}...","3/422-426 GLENFERRIE ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant..."
539051,-37.842220,145.033818,"[{'date': 'October 2007', 'price': '$1,700 pw'...","9 MERNDA ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant..."
539052,-37.841322,145.030886,"[{'date': 'February 2007', 'price': '$575 pw'}...","27 POWER AVENUE, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant..."
539053,-37.843799,145.032879,"[{'date': 'February 2007', 'price': '$350 pw'}...","699A TOORAK ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant..."


In [10]:
property_df['geometry'] = property_df.apply(lambda row: Point(row['lng'], row['lat']), axis=1)

In [7]:
school  = pd.read_csv(school_path)
school['geometry'] = school['geometry'].apply(wkt.loads)
gdf_school = gpd.GeoDataFrame(school, geometry='geometry')

In [8]:
gdf_school

,School_Name,geometry
0,Lockwood Primary School,"POLYGON ((144.19743 -36.82513, 144.20366 -36.8..."
1,Lockwood South Primary School,"POLYGON ((143.99198 -36.80865, 143.99179 -36.8..."
2,Heathcote Primary School,"POLYGON ((144.6649 -37.02629, 144.88917 -36.91..."
3,Redesdale Mia Mia Primary School,"POLYGON ((144.55801 -36.89947, 144.51209 -36.9..."
4,Axedale Primary School,"POLYGON ((144.71365 -36.74947, 144.66726 -36.7..."
...,...,...
1582,Matthew Flinders Girls Secondary College,"POLYGON ((144.4229 -38.08573, 144.41875 -38.08..."
1583,Canterbury Girls Secondary College,"POLYGON ((145.03764 -37.80765, 145.09942 -37.8..."
1584,Mentone Girls Secondary College,"POLYGON ((145.07547 -38.00074, 145.0713 -37.99..."
1585,Melbourne Girls College,"POLYGON ((145.02534 -37.79516, 145.04552 -37.8..."


In [40]:
gdf_houses_with_zones = gpd.sjoin(gpd.GeoDataFrame(df), gdf_school, how="inner", predicate="within")

In [43]:
# 1127421
in_zone_set = set(gdf_houses_with_zones.reset_index()['index'].values)

In [47]:
df.iloc[1].name

1

In [48]:
# gdf_houses_with_zones.reset_index()
df["inschool"] = df.reset_index().apply(lambda row: row.name in in_zone_set, axis=1)

In [52]:
df["inschool"].sum() / df.shape[0]

np.float64(0.9896411312389274)

In [ ]:
df

# PTV

In [29]:
paths = ["../data/raw/PTV/1 - Regional Train/stops_1_sa2.csv",
         "../data/raw/PTV/2 - Metropolitan Train/stops_2_sa2.csv",
         "../data/raw/PTV/3 - Metropolitan Tram/stops_3_sa2.csv", 
         "../data/raw/PTV/4 - Metropolitan Bus/stops_4_sa2.csv",
         "../data/raw/PTV/5 - Regional Coach/stops_5_sa2.csv",
         "../data/raw/PTV/6 - Regional Bus/stops_6_sa2.csv"]
total_ptv_coords = []
for path in paths:
    ptv = pd.read_csv(path)
    total_ptv_coords = total_ptv_coords + [ptv[['stop_lon','stop_lat']].values]
    # total_ptv_coords = total_ptv_coords + ptv[['stop_lon','stop_lat']].values\
total_ptv_coords = np.concatenate(total_ptv_coords)

In [30]:
ptv_df = pd.read_csv(paths[0])
ptv_df["geometry"] = ptv_df.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
ptv_gdf = gpd.GeoDataFrame(ptv_df, geometry='geometry')

In [31]:
num_to_find = 3
tree = cKDTree(total_ptv_coords)
property_coords = property_df[['lng', 'lat']].values
distances, indices = tree.query(property_coords, k=3)
property_df["ptv_index"] = list(indices)

In [32]:
def get_time_proximity(coordinates):
    url = "http://localhost:8080/ors/v2/directions/foot-walking"
    body = {"coordinates": coordinates}
    # print(body)
    response = requests.post(url, json=body)
    try:
        if response.status_code == 200:
            summary = response.json()["routes"][0]["summary"]
            if "duration" in summary:
                return summary["duration"]
            return 0
        else:
            if(response.json()["error"]["code"] == 2004):
                # distance is too long (> 100000.0 m)
                return -1
            if(response.json()["error"]["code"] == 2010):
                # "error":{"code":2010,"message":"Could not find routable point within a radius of 400.0 meters of specified coordinate
                return -1
            print(body)
            print(response.text)
            raise Exception("Error in request")
    except Exception as e:
        print(response.json())
        raise e

# # Example usage
# directions = get_time_proximity([[144.96332, -37.8140], [144.96332, -37.8120]])
# print(directions)

In [49]:
def get_time_proximity_from_property(row, ptv_stop_coords,tree):
    min_time = -1
    # if(num_to_find == 1):
    #     coordinates = [[row['lng'], row['lat']], ptv_stop_coords[indices].tolist()]
    #     return get_time_proximity(coordinates)
    for index in row['ptv_index']:
        coordinates = [[row['lng'], row['lat']], ptv_stop_coords[index].tolist()]
        time = get_time_proximity(coordinates)
        # print(time)
        if min_time == -1:
            min_time = time
        min_time = min(time,min_time)
    # coordinates = [[row['lng'], row['lat']], ptv_stop_coords[IndexError].tolist()]
    # return get_time_proximity(coordinates)
    # print(min_time)
    
    return min_time

In [58]:
i=5

path = f"../data/raw/oldlistings_rent_{i}.json"
property_df = pd.read_json(open(path, "r"))

# get index of the closest ptv stop
property_coords = property_df[['lng', 'lat']].values
distances, indices = tree.query(property_coords, k=3)
property_df["ptv_index"] = list(indices)

# dist time matching
property_df["time_to_ptv_1"] = property_df.progress_apply(lambda row: get_time_proximity_from_property(row, total_ptv_coords, tree), axis=1)
property_df.to_csv(f"../data/raw/property/rent_{i}_ptv.csv", index=False)

100%|██████████| 240134/240134 [33:56<00:00, 117.93it/s]


# join to orginal dataset

## read all

In [3]:
cleaned_df = pd.read_csv("../data/raw/property/rental_data_cleaned.csv")

In [6]:
cleaned_df['date'].drop_duplicates().sort_values()

4      2017-01-01
46     2017-02-01
21     2017-03-01
61     2017-04-01
18     2017-05-01
17     2017-06-01
20     2017-07-01
214    2017-08-01
19     2017-09-01
10     2017-10-01
96     2017-11-01
191    2017-12-01
16     2018-01-01
73     2018-02-01
56     2018-03-01
211    2018-04-01
144    2018-05-01
123    2018-06-01
38     2018-07-01
15     2018-08-01
93     2018-09-01
14     2018-10-01
70     2018-11-01
143    2018-12-01
142    2019-01-01
55     2019-02-01
72     2019-03-01
80     2019-04-01
118    2019-05-01
282    2019-06-01
84     2019-07-01
13     2019-08-01
7      2019-09-01
107    2019-10-01
278    2019-11-01
257    2019-12-01
65     2020-01-01
63     2020-02-01
129    2020-03-01
128    2020-04-01
89     2020-05-01
2      2020-06-01
5      2020-07-01
32     2020-08-01
9      2020-09-01
8      2020-10-01
26     2020-11-01
25     2020-12-01
48     2021-01-01
24     2021-02-01
62     2021-03-01
22     2021-04-01
43     2021-05-01
1      2021-06-01
6      2021-07-01
0      202

In [5]:
# read all the ptv data
all_ptv = pd.concat([pd.read_csv(f"../data/raw/property/rent_{i}_ptv.csv") for i in range(1,6)])

In [10]:
cleaned_df

,lat,lng,address,bed,bath,car,type,rented_price,date,year,geometry,sa2_code_left,sa2_name_left,population,sa2_code_right,region,median_income,cpi,unemployment_rate
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,540,2021-08-01,2021,POINT (144.95558001 -37.81373001),206041505.0,Melbourne CBD - West,16098.0,206041505.0,Melbourne CBD - West,39300.0,3.5,4.200000
1,-37.810280,144.956670,"2007/228 ABECKETT STREET, MELBOURNE",2.0,2.0,0.0,Unit/apmt,360,2021-06-01,2021,POINT (144.95667001 -37.81028001),206041504.0,Melbourne CBD - North,17360.0,206041504.0,Melbourne CBD - North,29100.0,3.5,4.200000
2,-37.810280,144.956670,"2007/228 ABECKETT STREET, MELBOURNE",2.0,2.0,0.0,Unit/apmt,660,2020-06-01,2020,POINT (144.95667001 -37.81028001),206041504.0,Melbourne CBD - North,19155.0,206041504.0,Melbourne CBD - North,25000.0,0.9,5.166667
3,-37.809259,144.966567,"1808/318 RUSSELL STREET, MELBOURNE",2.0,2.0,0.0,Unit/apmt,650,2021-06-01,2021,POINT (144.9665671 -37.809259),206041503.0,Melbourne CBD - East,10119.0,206041503.0,Melbourne CBD - East,40810.0,3.5,4.200000
4,-37.809259,144.966567,"1808/318 RUSSELL STREET, MELBOURNE",2.0,2.0,0.0,Unit/apmt,720,2017-01-01,2017,POINT (144.9665671 -37.809259),206041503.0,Melbourne CBD - East,11881.0,206041503.0,Melbourne CBD - East,31790.0,1.9,5.608333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132488,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-09-01,2019,POINT (144.73271 -37.883631),213051581.0,Point Cook - North West,9739.0,213051581.0,Point Cook - North West,57433.0,1.8,5.175000
1132489,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-08-01,2019,POINT (144.73271 -37.883631),213051581.0,Point Cook - North West,9739.0,213051581.0,Point Cook - North West,57433.0,1.8,5.175000
1132490,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-07-01,2019,POINT (144.73271 -37.883631),213051581.0,Point Cook - North West,9739.0,213051581.0,Point Cook - North West,57433.0,1.8,5.175000
1132491,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,365,2018-06-01,2018,POINT (144.73271 -37.883631),213051581.0,Point Cook - North West,9678.0,213051581.0,Point Cook - North West,57001.0,1.8,5.283333


In [17]:
all_ptv['ptv_time'] = all_ptv['time_to_ptv_1']
all_ptv = all_ptv[['lat','lng','address','ptv_time']]
all

In [37]:
all_ptv = all_ptv.drop_duplicates()

In [38]:
merged_df = cleaned_df.merge(all_ptv, on=['lat','lng','address'], how='left')

In [39]:
merged_df.shape

(1132493, 20)

In [21]:
cleaned_df.shape

(1132493, 19)

In [24]:
unmatched_rows = merged_df[merged_df['ptv_time'].isna()]

In [33]:
duplicated_rows = all_ptv[all_ptv.duplicated(subset=['lat', 'lng', 'address'], keep=False)]

In [36]:
all_ptv.drop_duplicates().shape

(1860717, 4)

In [41]:
merged_df.to_csv("../data/raw/property/rental_data_cleaned_ptv.csv", index=False)

# Store Data in forms of a mapping

In [2]:
data = pd.read_csv("../data/raw/property/rental_data_cleaned_ptv.csv")

In [5]:
data.shape

(1132493, 20)

In [6]:
data[['ptv_time','lat','lng']].drop_duplicates().to_csv("../data/raw/time_ptv.csv", index=False)